In [ ]:
!pip install neo4j

Connect to the Neo4j Database

In [1]:
from neo4j import GraphDatabase

# Connection details
uri = "neo4j+s://878e8d59.databases.neo4j.io"
username = "neo4j"
password = "uvpmwHTcfx6OafM2JQpMfy05Jblkvft3YTOyDib2osU"

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Test the connection
def test_connection(driver):
    with driver.session() as session:
        result = session.run("RETURN 'Connection Successful!'")
        print(result.single()[0])

test_connection(driver)


Connection Successful!


 Design the Graph Schema

In [5]:
def create_graph_schema(driver):
    with driver.session() as session:
        # Create Department nodes
        session.run("CREATE (:Department {department_name: 'HR', department_id: 'D1'})")
        session.run("CREATE (:Department {department_name: 'Engineering', department_id: 'D2'})")

        # Create Employee nodes
        session.run("""
            CREATE (e1:Employee {name: 'Alice', employee_id: 'E1', position: 'Manager', department: 'HR', email: 'alice@example.com'})
        """)
        session.run("""
            CREATE (e2:Employee {name: 'Bob', employee_id: 'E2', position: 'Engineer', department: 'Engineering', email: 'bob@example.com'})
        """)
        session.run("""
            CREATE (e3:Employee {name: 'Charlie', employee_id: 'E3', position: 'HR Specialist', department: 'HR', email: 'charlie@example.com'})
        """)

        # Create BELONGS_TO relationships
        session.run("""
            MATCH (e:Employee {name: 'Alice'}), (d:Department {department_name: 'HR'})
            CREATE (e)-[:BELONGS_TO]->(d)
        """)
        session.run("""
            MATCH (e:Employee {name: 'Bob'}), (d:Department {department_name: 'Engineering'})
            CREATE (e)-[:BELONGS_TO]->(d)
        """)
        session.run("""
            MATCH (e:Employee {name: 'Charlie'}), (d:Department {department_name: 'HR'})
            CREATE (e)-[:BELONGS_TO]->(d)
        """)

        # Create REPORTS_TO relationships
        session.run("""
            MATCH (e1:Employee {name: 'Charlie'}), (e2:Employee {name: 'Alice'})
            CREATE (e1)-[:REPORTS_TO]->(e2)
        """)
        session.run("""
            MATCH (e2:Employee {name: 'Bob'}), (e3:Employee {name: 'Alice'})
            CREATE (e2)-[:REPORTS_TO]->(e3)
        """)

create_graph_schema(driver)


Create and Query Nodes and Relationships

In [7]:
def find_subordinates(driver, manager_name):
    with driver.session() as session:
        query = """
        MATCH (manager:Employee {name: $manager_name})<-[:REPORTS_TO]-(subordinate:Employee)
        RETURN subordinate.name AS subordinate_name
        """
        results = session.run(query, manager_name=manager_name)
        for record in results:
            print(record['subordinate_name'])

# Find all subordinates of 'Alice'
find_subordinates(driver, "Alice")


Bob
Charlie
Bob
Charlie
Bob
Charlie
Bob
Charlie


Indexing for Performance

In [9]:
def create_indexes(driver):
    with driver.session() as session:
        session.run("CREATE INDEX FOR (e:Employee) ON (e.employee_id)")
        session.run("CREATE INDEX FOR (d:Department) ON (d.department_id)")

create_indexes(driver)
